<a href="https://colab.research.google.com/github/buwuyou/CTCN_2020_EE/blob/master/CTCN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook demostrates how to set up [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) in dealing with satellite imagery. Specifically, this notebook shows:
1. Authentication Earth Engine Python API to this server
2. Define Area of Interest for Earth Engine Datasets
3. Load satellite imagery, process and display
4. Export image


[Earth Engine Python API](https://developers.google.com/earth-engine/python_install) facilitates interacting with Earth Engine servers using the Python programming language. As compared to Earth Engine Javascript Playground (https://code.earthengine.google.com/), Python API allows the user to use jupyter notebook and rich sources of Python libraries, which are particular good for training purposes. The main difference is the syntex-wise definition of variable and function according to Python lauguage. The core of Earth Engine API function is persistent (https://developers.google.com/earth-engine/api_docs). Unlike Earth Engine Python API, there are a rich resources of tutorials to learn Earth Engine Javascript API, check here to learn more: https://developers.google.com/earth-engine/tutorials

Prerequisites: Review the [Earth Engine get started guide](https://developers.google.com/earth-engine/getstarted) and [Earth Engine Python API doc](https://developers.google.com/earth-engine/python_install)

***TIP:*** 

**You do not need to understand everything (at least not right now)**. Your goal is to run through the tutorial end-to-end and get results. You do not need to understand everything on the first pass. List down your questions as you go. Make heavy use of the [API documentation](https://developers.google.com/earth-engine/getstarted) to learn about all of the functions that you’re using.


# 1. Authentificate GEE Python API

Make sure you sign up for a GEE account beforehand: https://signup.earthengine.google.com/#!/

In [0]:
#@title
!pip install -q earthengine-api

# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

Import python libraies for map visualization (folium: https://python-visualization.github.io/folium/), pretty printer for printing information of earth engine objects (pprint: https://docs.python.org/3/library/pprint.html)

In [0]:
#@title
import folium
from pprint import pprint 
from folium import plugins
print(folium.__version__)

Add custom basemaps, generic setup

In [0]:
#@title
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# 2. Define Area of Interest

Earth Engine handles vector data with the Geometry type, including Point (a list of coordinates in some projection), LineString (a list of points), LinearRing (a closed LineString), and Polygon (a list of LinearRings where the first is a shell and subsequent rings are holes). Check here to learn more: https://developers.google.com/earth-engine/geometries

We will define a Geometry as Polygon, which defined coordinates as a GeoJSON polygon. To get the coordinates in a simple way, this is a website to define GeoJSON geometry (http://geojson.io/), draw a polygon on the map using the web-tool, then you will find the coordinates on the right panel, replace the coordinates below according to the coordinates of your AOI.


In [0]:
AOI = ee.Geometry.Polygon(
        [[[
              89.40673828125,
              21.37124437061831
            ],
            [
              90.90087890624999,
              21.37124437061831
            ],
            [
              90.90087890624999,
              22.471954507739227
            ],
            [
              89.40673828125,
              22.471954507739227
            ],
            [
              89.40673828125,
              21.37124437061831
            ]]])

## Display the defined AOI

In [0]:
map = folium.Map(location=[22., 89.], zoom_start=7)
folium.GeoJson(
            data = AOI.getInfo(),
            name = 'Area of Interest',
            overlay = True,
            control = True
        ).add_to(map)
map.add_child(folium.LayerControl())
map

# 3. Load satellite imagery, process and display them
This section includes:
0. General introduction on how to find and display data from earth engine
1. Landsat-8 surface reflectance dataset display, check metadata and cloud masking
2. Sentinel-2 surface reflectance dataset display, check metadata and cloud masking
3. Sentinel-1 backscatter display and check metadata

## Search for datasets in Earth Engine Data Catalog

Use the provided link below: 
https://developers.google.com/earth-engine/datasets

Try to search for 'elevation' or 'dem' or 'SRTM', you will see a result as 'SRTM Digital Elevation Data 30m' , click and read the dataset descriptions to understand the format. The '**Earth Engine Snippet**' indicates the ID for this dataset. The ID for this DEM image is 
>`ee.Image("USGS/SRTMGL1_003")`

We will load this image and find out the elevation for the Mount Everest.

In [0]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

### SRTM DEM 30m

In [0]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
mapid = dem.updateMask(dem.gt(0)).getMapId(vis_params)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

Again, try to search for 'landsat' and 'landsat 7' and 'landsat 8 surface reflectance', understand the differences in Landsat long-term time series data.

### Landsat-8

Load the dataset, filter the images within the AOI, and filter the acquring datas for Jan, 2019

In [0]:
# Use Landsat 8 surface reflectance data.
l8sr = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
            .filterBounds(AOI)
            .filterDate('2019-01-01', '2019-12-31')
            .filter(ee.Filter.lt('CLOUD_COVER',50))
            .sort('CLOUD_COVER',False))

In [0]:
pprint ({'Number of landsat images': l8sr.size().getInfo()})

In [0]:
pprint ({'First landsat image': l8sr.first().getInfo()})

Display the image with clouds

In [0]:
# The image input data is a 2019 cloud-masked median composite.
image = l8sr.first()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
map = folium.Map(location=[22.,89.], zoom_start=7)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat-8 with clouds',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Define a function for cloud and shadow masking

In [0]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

In [0]:
# The image input data is a 2019 cloud-masked image.
image = l8sr.map(maskL8sr).first()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[22.,89.], zoom_start=7)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat-8 without clouds',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Display the median composite image and clip to AOI

In [0]:
# The image input data is a 2019 cloud-masked median composite.
image = l8sr.map(maskL8sr).median().clip(AOI)

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[22.,89.], zoom_start=8)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='L8 median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Sentinel-2

In [0]:
# Function to mask clouds using the Sentinel-2 SCL band.
def S2CloudMaskSCL(img):
  qa = img.select('SCL')
  mask_band = qa.eq(3).Or(qa.eq(8)).Or(qa.eq(9)).Or(qa.eq(10)).Or(qa.eq(11))
  mask = mask_band.eq(0)
  # Return the masked and scaled data.
  return img.updateMask(mask)
def ESAcloudMask(img):
    ''' Sentinel-2 Bits 10 & 11 are clouds & cirrus, so set to 0. '''
    qa = img.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(clear) 

In [0]:
s2sr = (ee.ImageCollection('COPERNICUS/S2_SR')
                      .filterDate('2019-01-01', '2019-12-31')
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                      .filterBounds(AOI)
                      .map(S2CloudMaskSCL)
                      .map(ESAcloudMask))



In [0]:
pprint ({'Number of Sentinel-2 images': s2sr.size().getInfo()})

In [0]:
pprint ({'First Sentinel-2 image': s2sr.first().getInfo()})

In [0]:
# The image input data is a 2019 cloud-masked median composite.
s2image = s2sr.median().clip(AOI)
l8image = l8sr.map(maskL8sr).median().clip(AOI)
# Use folium to visualize the imagery.
mapIdDict = s2image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
mapIdDict1 = l8image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[22.,89.], zoom_start=8)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S2 median composite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='L8 median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Sentinel-1

In [0]:
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
  .filterBounds(AOI)
  .filter(ee.Filter.And(
    ee.Filter.date('2019-01-01', '2019-01-31'),
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'),
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')
  )))


In [0]:
orbit = ['ASCENDING','DESCENDING']  
s1asc = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit[0])).first()
pprint ({'print s1 ascending image info:': s1asc.getInfo()})

In [0]:
s1des = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit[1])).first()
pprint ({'print s1 descending image info:': s1des.getInfo()})

In [0]:
mapIdDict = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit[0])).median().getMapId({'bands': ['VV'], 'min': -20, 'max': 4})
mapIdDict1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit[1])).median().getMapId({'bands': ['VV'], 'min': -20, 'max': 4})
map = folium.Map(location=[22.,89.], zoom_start=8)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S1 Ascending',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S1 Descending',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
s2image = s2sr.median().clip(AOI)
l8image = l8sr.map(maskL8sr).median().clip(AOI)
s1image = s1.median().clip(AOI)
s1rgb = s1.map(lambda image: image.addBands(ee.Image(image.select('VV').divide(image.select('VH')).rename('ratio')))).median().clip(AOI)
# Use folium to visualize the imagery.
mapIdDict = s2image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
mapIdDict1 = l8image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
mapIdDict2 = s1image.getMapId({'bands': ['VV'], 'min': -20, 'max': 4})
mapIdDict3 = s1rgb.getMapId({'bands':['VV','VH','ratio'],'min':[-20,-28,0],'max':[4,1,2]})
map = folium.Map(location=[22.,89.], zoom_start=8)
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S2 median composite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='L8 median composite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S1 VV median composite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapIdDict3['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='S1 RGB median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Export image

Export one Sentinel-2 RGB image and check it in your google drive when the task has been completed.
**It will take a bit time for a cup of tea!**

In [0]:
# define a smaller aoi for exporting
export_aoi = ee.Geometry.Polygon([
          [
            [
              89.57324981689453,
              22.354203655362834
            ],
            [
              89.70096588134764,
              22.354203655362834
            ],
            [
              89.70096588134764,
              22.4300707662349
            ],
            [
              89.57324981689453,
              22.4300707662349
            ],
            [
              89.57324981689453,
              22.354203655362834
            ]
          ]
        ])
task= ee.batch.Export.image.toDrive(
            image = s2image.select(['B4', 'B3', 'B2']),
            description = f'CTCN101',
            folder = f'CTCN101',
            region= export_aoi.getInfo()["coordinates"],    
            scale= 10,
            maxPixels= int(2e9),
            crs='EPSG:4326')
task.start()

# Block until the task completes.
print('Running image export to Google Drive...')
import time
while task.active():
  time.sleep(30)

# Error condition
if task.status()['state'] != 'COMPLETED':
  print('Error with image export.')
else:
  print('Image export completed.')

> Try to export one image of Sentinel-1 reusing the code above?

Find the export image in your Google Drive, download it and display in your geospatial data analysis software, such as ArcGIS, QGIS

# Review what we learned

Try to search for Earth Engine functions we made use from above materials, which starts with 
>`ee.`

And find and understand the funtion description using the API docs: https://developers.google.com/earth-engine/api_docs